## 1.Scraping the Dataset of Toronto Neighborhood


The dataset of the Toronto neighborhood is not available in csv form for manipulation, therefore its data should be scraped from wikipedia here is the technique with step by step guide of how to scrap the data.
The Beautiful Soup package is used for scraping the data.

Before getting start lets download the following libraries

In [1]:
from bs4 import BeautifulSoup # to scraping the site and getting required data
import requests
import pandas as pd
import numpy as np

 Getting the request and parsing 

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpVnAwpAAEYAAEsuV0QAAADD","wgCSPNonce":!1,"wgCanonica

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source,'lxml')


If you carefully inspect the HTML script all the table contents i.e. postalcode,borough and neighborhood of the toronto which we intend to extract is under class Wikitable .

In [5]:
my_table=soup.find('table',{'class':'wikitable'})#find the class where required data is place


Now after examing the data we sure our required data lies in a step from 'tbody','tr' to the element of 'td'.Creating a list for storing our data.

In [6]:

data=[]
table_body=my_table.find('tbody')
sub_table=table_body.find_all('tr')
for row in sub_table:
    sub=row.find_all('td') # data resides in the element of td
    sub=[ele.text.strip() for ele in sub]
    data.append(sub)



### Transform the data into Pandas dataframe  

From the links, we have to extract our required data which is postalcode,borough and neighborhood.
Now Convert the following data into Pandas DataFrame to work in python.


In [11]:
arr=np.array(data)
arr_refine=[x for x in arr if x] #removing the empty list

postal_code=[index[0] for index in arr_refine] #getting the postalcode
borough=[index[1] for index in arr_refine] #getting the borough
neighbor=[index[2] for index in arr_refine] #getting the neighorhood


 
df = pd.DataFrame(list(zip(postal_code,borough,neighbor)), #creating a table of following columns
               columns =['Postal Code','Borough','Neighborhood'])


df_refine=df[df.Borough !='Not assigned'].reset_index(drop=True) # as required ignore the Not assigned cells

toronto_data=df_refine.apply(lambda x: x.str.replace('/',',')) #as required add comma

toronto_data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


For calculating row lenght of our data

In [8]:
toronto_data.shape[0]

103

## 2. Adding the Location(Latitude & Longitude) in Data set

Here the csv file data set location is used you can get it from Link :http://cocl.us/Geospatial_data

In [12]:
location=pd.read_csv('Geospatial_Coordinates.csv')
toronto_data_location=pd.merge(toronto_data,location, on=['Postal Code']) #Alert on merging columns name should be same
toronto_data_location

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
